In [1]:
#!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
weather_df = pd.read_csv("c://user/ilariatare/Desktop/Class/weather_df.csv", encoding="utf-8")

In [3]:
weather_df.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Hilo,US,1617063588,19.7297,-155.0900,78.8,61,40,6.91
1,1,Clyde River,CA,1617063588,70.4692,-68.5914,-7.6,84,90,16.11
2,2,Geraldton,AU,1617063589,-28.7667,114.6000,75.2,69,75,9.22
3,3,Baní,DO,1617063589,18.2833,-70.3333,81.0,77,3,11.39
4,4,Bambous Virieux,MU,1617063589,-20.3428,57.7575,75.0,94,20,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Heatmap of humidity
humidity = weather_df["Humidity"]
max_humidity = humidity.max()
locations = weather_df[["Latitude", "Longitude"]]

In [5]:
# Configure gmaps
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temperature"] >= 70) & (weather_df["Max Temperature"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
24,24,Upington,ZA,1617063135,-28.4478,21.2561,75.20,38,0,8.05
48,48,Saint-Philippe,RE,1617063610,-21.3585,55.7679,77.00,73,0,5.75
217,217,Saint-Joseph,RE,1617063671,-21.3667,55.6167,77.00,73,0,5.75
224,224,Buin,CL,1617063673,-33.7333,-70.7500,70.00,67,0,9.22
379,379,Caravelas,BR,1617063740,-17.7125,-39.2481,77.95,73,0,7.65
447,447,Gouré,NE,1617063763,13.9835,10.2704,75.00,10,0,10.00
449,449,Melipilla,CL,1617063764,-33.7000,-71.2167,70.00,94,0,2.30
509,509,Dubai,AE,1617063664,25.2582,55.3047,70.00,83,0,6.91
551,551,Saint-Paul,RE,1617063619,-21.0096,55.2707,73.40,94,0,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrowed_city_df.reindex(["City","Country", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""

hotel_df

#unclear why i'm getting "NaN" for the inputs 

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
Country,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
Latitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
Longitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
 
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
  
    params["location"] = f"{lat},{lng}"

    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index City: nan.
Missing field/result... skipping.
------------
Retrieving Results for Index Country: nan.
Missing field/result... skipping.
------------
Retrieving Results for Index Latitude: nan.
Missing field/result... skipping.
------------
Retrieving Results for Index Longitude: nan.
Missing field/result... skipping.
------------


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [10]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig


IndexError: list index out of range